In [40]:
# pip install pandas
# pip install plotly
# pip install --upgrade nbformat

import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import psycopg2


In [42]:
conn = psycopg2.connect(**db_config)
cursor = conn.cursor()

In [59]:
## Consultando a Tabela tb_gpx_full ##
consulta_gpx_full = "SELECT * FROM tb_gpx_full"
cursor.execute(consulta_gpx_full)
resultados_gpx_full = cursor.fetchall()
df_gpx_full = pd.DataFrame(resultados_gpx_full, columns=[desc[0] for desc in cursor.description])
df_gpx_full.head()

,id_rota,nome_usuario,latitude,longitude,elevacao,data_rota,hora_rota,cidade,estado,pais
0,10129297,LEXUS_RX_HYBRID_2020,37.15449769459162,-121.6656713874618,86.867,2023-09-10,20:20:28,Santa Clara County,US-CA,US
1,10129297,LEXUS_RX_HYBRID_2020,37.1544975923951,-121.66567157676032,86.858,2023-09-10,20:20:28,Santa Clara County,US-CA,US
2,10129297,LEXUS_RX_HYBRID_2020,37.15449748543151,-121.66567178310542,86.862,2023-09-10,20:20:28,Santa Clara County,US-CA,US
3,10129297,LEXUS_RX_HYBRID_2020,37.15449737825824,-121.6656720035424,86.868,2023-09-10,20:20:29,Santa Clara County,US-CA,US
4,10129297,LEXUS_RX_HYBRID_2020,37.15449726977479,-121.66567223442958,86.91,2023-09-10,20:20:29,Santa Clara County,US-CA,US


In [60]:
## Consultando a Tabela tb_gpx_full ##
consulta_gpx_distancias = '''
SELECT 
    id_rota, nome_usuario, inicio_rota, fim_rota, tempo_viagem, cidade, estado, pais, dist_km
FROM (
    SELECT 
        id_rota, nome_usuario, inicio_rota, fim_rota, tempo_viagem, cidade, estado, pais, dist_km,
        ROW_NUMBER() OVER (PARTITION BY nome_usuario ORDER BY dist_km DESC) AS row_num
    FROM VW_DISTANCIAS_PERCORRIDAS
    WHERE fim_rota IS NOT NULL
) AS ranked_data
WHERE row_num = 1
ORDER BY dist_km DESC
limit 5
;
'''

cursor.execute(consulta_gpx_distancias)
resultados_gpx_distancias = cursor.fetchall()
df_gpx_distancias = pd.DataFrame(resultados_gpx_distancias, columns=[desc[0] for desc in cursor.description])  
df_gpx_distancias.head()

,id_rota,nome_usuario,inicio_rota,fim_rota,tempo_viagem,cidade,estado,pais,dist_km
0,10189064,TOYOTA_RAV4_HYBRID_2019,08:12:53,08:22:53,0 days 00:10:00,Natrona County,US-WY,US,21.111025
1,10129303,HYUNDAI_SONATA_HYBRID_2021,20:03:21,20:13:20,0 days 00:09:59,Ford County,US-IL,US,19.267374
2,10176193,TOYOTA_RAV4_2019,22:53:14,23:03:14,0 days 00:10:00,DuPage County,US-IL,US,18.495468
3,10129455,LEXUS_RX_HYBRID_2020,04:58:20,05:08:20,0 days 00:10:00,Los Angeles County,US-CA,US,18.223248
4,10189055,TOYOTA_PRIUS_v_2017,07:52:53,08:02:53,0 days 00:10:00,Natrona County,US-WY,US,18.192307


In [ ]:


############################################
### GERANDO GRAFICO DE VIAGENS EFETUADAS ###
############################################
fig1 = px.bar(
    df_gpx_full_maps,
    x='nome_usuario',
    y='dist_km',
    text='dist_km',
    title='As 5 Viagens Mais Longas por Motorista',
    labels={'nome_usuario': 'Motorista', 'dist_km': 'Distância (km)'},
    color='nome_usuario'
)

#fig1.update_traces(texttemplate='%{text}', textposition='outside')
fig1.update_traces(texttemplate='%{text}', textposition='inside')

fig1.show()

In [61]:

 ## Consultando a Tabela tb_gpx_full ##
consulta_maps = '''
 
 SELECT b.id_rota, b.nome_usuario, b.data_rota,b.inicio_rota, b.fim_rota, b.tempo_viagem, b.cidade, b.estado, b.pais, b.dist_km
    FROM (
        SELECT
            id_rota,data_rota, nome_usuario, inicio_rota, fim_rota, tempo_viagem, cidade, estado, pais, dist_km
        FROM (
            SELECT
                id_rota, nome_usuario, data_rota,inicio_rota, fim_rota, tempo_viagem, cidade, estado, pais, dist_km,
                ROW_NUMBER() OVER (PARTITION BY nome_usuario ORDER BY dist_km DESC) AS row_num
            FROM VW_DISTANCIAS_PERCORRIDAS
            WHERE fim_rota IS NOT NULL
        ) AS ranked_data
        WHERE row_num = 1
        ORDER BY dist_km DESC
        LIMIT 5
    ) AS b
 inner join tb_rotas as a on a.id_rota = b.id_rota
 '''

cursor.execute(consulta_maps)
resultados_maps = cursor.fetchall()
df_maps = pd.DataFrame(resultados_maps, columns=[desc[0] for desc in cursor.description])  
df_maps.head()

,id_rota,nome_usuario,data_rota,inicio_rota,fim_rota,tempo_viagem,cidade,estado,pais,dist_km
0,10176193,TOYOTA_RAV4_2019,2023-09-13,22:53:14,23:03:14,0 days 00:10:00,DuPage County,US-IL,US,18.495468
1,10129303,HYUNDAI_SONATA_HYBRID_2021,2023-09-07,20:03:21,20:13:20,0 days 00:09:59,Ford County,US-IL,US,19.267374
2,10129455,LEXUS_RX_HYBRID_2020,2023-09-11,04:58:20,05:08:20,0 days 00:10:00,Los Angeles County,US-CA,US,18.223248


In [ ]:
fig = px.line_mapbox(
    df_maps,
    lat='latitude',  # Coluna contendo latitudes
    lon='longitude',  # Coluna contendo longitudes
    text='nome_usuario',  # Rótulos a serem exibidos nas rotas (nome do usuário)
    title='Mapa de Rotas dos 5 Motoristas',
    hover_name='nome_usuario',  # Informações adicionais ao passar o mouse sobre as rotas
    line_group='nome_usuario',  # Agrupar rotas por nome do usuário
    zoom=5  # Nível de zoom do mapa
)

# Configurar o layout do mapa
fig.update_layout(mapbox_style="carto-positron")
fig.update_layout(mapbox_center={"lat": df_maps['latitude'].mean(), "lon": df_maps['longitude'].mean()})

# Mostrar o mapa interativo
fig.show()

In [63]:
ids_rotas = df_gpx_distancias['id_rota'].tolist()
ids_rotas
df_gpx_full_maps = df_gpx_full[df_gpx_full['id_rota'].isin(ids_rotas)]
df_gpx_full_maps

,id_rota,nome_usuario,latitude,longitude,elevacao,data_rota,hora_rota,cidade,estado,pais
14653,10129303,HYUNDAI_SONATA_HYBRID_2021,40.32601646702934,-88.17708621910162,195.889,2023-09-07,20:03:21,Ford County,US-IL,US
14654,10129303,HYUNDAI_SONATA_HYBRID_2021,40.32602400811869,-88.17708028199617,195.886,2023-09-07,20:03:21,Ford County,US-IL,US
14655,10129303,HYUNDAI_SONATA_HYBRID_2021,40.32604498504896,-88.17706347068096,195.906,2023-09-07,20:03:21,Ford County,US-IL,US
14656,10129303,HYUNDAI_SONATA_HYBRID_2021,40.32605990435968,-88.17705162298773,195.933,2023-09-07,20:03:21,Ford County,US-IL,US
14657,10129303,HYUNDAI_SONATA_HYBRID_2021,40.326073615348086,-88.17704075522732,195.946,2023-09-07,20:03:21,Ford County,US-IL,US
...,...,...,...,...,...,...,...,...,...,...
609169,10189064,TOYOTA_RAV4_HYBRID_2019,42.93663682109088,-106.33407775029838,1632.923,2023-09-12,08:22:52,Natrona County,US-WY,US
609170,10189064,TOYOTA_RAV4_HYBRID_2019,42.93660745389394,-106.33408242945072,1632.854,2023-09-12,08:22:53,Natrona County,US-WY,US
609171,10189064,TOYOTA_RAV4_HYBRID_2019,42.93657763835444,-106.33408710011754,1632.818,2023-09-12,08:22:53,Natrona County,US-WY,US
609172,10189064,TOYOTA_RAV4_HYBRID_2019,42.93654753178426,-106.33409186306037,1632.758,2023-09-12,08:22:53,Natrona County,US-WY,US


In [65]:
fig = px.line_mapbox(
    df_gpx_full_maps,
    lat='latitude',  # Coluna contendo latitudes
    lon='longitude',  # Coluna contendo longitudes
    text='nome_usuario',  # Rótulos a serem exibidos nas rotas (nome do usuário)
    title='Mapa de Rotas dos 5 Motoristas',
    hover_name='nome_usuario',  # Informações adicionais ao passar o mouse sobre as rotas
    line_group='nome_usuario',  # Agrupar rotas por nome do usuário
    zoom=5  # Nível de zoom do mapa
)

# Configurar o layout do mapa
fig.update_layout(mapbox_style="carto-positron")
fig.update_layout(mapbox_center={"lat": df_maps['latitude'].mean(), "lon": df_maps['longitude'].mean()})

# Mostrar o mapa interativo
fig.show()

TypeError: Could not convert string '40.3260164670293440.3260240081186940.3260449850489640.3260599043596840.32607361534808640.3260885886880440.3261024538760940.3261174631823240.3261326015066340.3261464596555440.32616179470354540.3261767802651640.3261906805229940.32620603072836540.3262201231129240.3262357418031940.3262496949671940.3262640863773740.326279028391640.3262940769790740.3263088932746440.32632396726940.3263389183906840.3263538038787140.3263687618590940.3263837138677840.3263988146048440.3264139243486440.32642906092810640.3264440627938840.3264589651488540.3264738026929340.32648900378650640.326504358715640.326521516071540.3265363954521440.32655260526464540.3265686653556540.32658341281836640.32659958679840.3266073748998140.3266302867923740.32664617909800640.3266608536281640.3266765845098940.326690579640.3267061258080740.3267220140187940.3267362845289640.326752138416240.3267678007409140.3267826531669240.32679106356414440.326813525506240.32682948310528640.3268441344129840.32685944699